Jupyter# **<center>SPARC FAIR Codeathon 2022</center>**
<center>
<a href="https://sparc.science">
<img src="https://sparc.science/_nuxt/img/logo-sparc-wave-primary.8ed83a5.svg" alt="SPARC" width="150"/>
</a>
</center>
<center>
<a href="https://sparc.science/help/2022-sparc-fair-codeathon">
<img src="https://images.ctfassets.net/6bya4tyw8399/2qgsOmFnm7wYIfRrPrqbgx/ae3255858aa12bfcebb52e95c7cacffe/codeathon-graphic.png" alt="FAIR" width="75">
</a>
</center>

## <center>Tutorial 1: Mapping 2D data to a 3D organ scaffold</center>


## **Introduction**
Welcome to the first of the Quilted Tutorials! We will be demonstrating different features from the [**SPARC**](https://sparc.science/) project. The goal will be to project the 2D locations of neurites in the rat stomach onto a 3D scaffold of the organ. The data points and the 3D scaffold will be pulled from **SPARC** datasets. This workflow could be applied to datasets for other organ systems. 

Because the data is [**FAIR**](https://www.nature.com/articles/sdata201618) we can easily re-use and combine three different datasets of the spatial distribution of afferent and efferent vagal neurons. 

Here is the workflow for this tutorial:

![workflow](img/workflow.png)



## **Installing the dependencies**
This tutorial relies on several Python packages that have been developed as part of the **SPARC** project. We will be installing them in order to complete this tutorial.

In [ ]:
!pip install -r requirements.txt

### **Imports**
Here we import all of the required dependencies.

In [ ]:
import os
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from stl import mesh as msh
from tqdm import tqdm
from zipfile import ZipFile
from mpl_toolkits import mplot3d
from ipywidgets import interact, Checkbox, fixed

## **Retrieving the data**
Now that all the dependencies have been installed we will retrieve the data directly from the [**SPARC**](https://sparc.science) project website using the Pennsieve API for the SPARC portal. 

We will be using the following three datasets:
 * [Vagal afferents associated with the myenteric plexus of the rat stomach](https://sparc.science/datasets/10?type=dataset&datasetDetailsTab=files)
 * [Vagal afferents within the longitudinal and circular muscle layers of the rat stomach](https://sparc.science/datasets/11?type=dataset&datasetDetailsTab=files)
 * [Vagal efferents associated with the myenteric plexus of the rat stomach](https://sparc.science/datasets/12?type=dataset&datasetDetailsTab=files)
 
We'll first define a few helper functions to search, display and download datasets within this notebook:

In [ ]:
def search_dataset(query, limit=5):
    """ Searches the SPARC data portal for the given query
        Inputs: 
        query -- string to search as a keyword in the dataset
        limit -- integer limit for the number of results to return, defualt 5
        
        Outputs:
        rst -- string of concatenated json tags for return results with the id, version, name and tags fields only for all returned results
        
    """
    url = "https://api.pennsieve.io/discover/search/datasets?limit="+str(limit)+"&offset=0&query="+query+"&orderBy=relevance&orderDirection=desc"
    headers = {"Accept": "application/json"}
    response = requests.get(url, headers=headers)
    rst = []
    for r in response.json()['datasets']:
        rst += [{'id':r['id'], 'version':r['version'], 'name':r['name'], 'tags':r['tags']}]
    return rst

def print_folder_structure(dataId, version, max_level=3): # taken from stackoverflow
    """ Print the directory structure of a dataset to the console output. This assumes that it is saved in the root directory with default filename.
    
    Inputs: 
    dataId -- integer id of the result
    version -- integer dataset version 
    max_level -- integer depth of directory structure to return, default 3
    
    Outputs: 
    None 
    """
    startpath = "Pennsieve-dataset-"+str(dataId)+"-version-"+str(version)
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        if level == max_level: break
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))

def get_dataset(dataId, version, dest_dir="."):
    """ Save a dataset from the SPARC data portal using the Pennsieve API.
    
    Inputs: 
    dataId -- integer id of the dataset
    version -- integer dataset version 
    dest_dir -- string directory to save data set into. Default is root.
    
    Outputs: 
    None 
    """
    url = "https://api.pennsieve.io/discover/datasets/"+str(dataId)+"/versions/"+str(version)+"/download?"
    # download dataset
    response = requests.get(url, stream = True)
    file_zip = "data.zip"
    data_file = open(file_zip,"wb")
    for chunk in tqdm(response.iter_content(chunk_size=1024)):
        data_file.write(chunk)
    data_file.close()
    # unzip dataset
    with ZipFile(file_zip, 'r') as obj:
       obj.extractall()
    # delete temporary zip file
    os.remove(file_zip)

The following line searches for the data sets we are interested in (keyword 'vagal') and prints out the search results:

In [ ]:
search_dataset('vagal')

Next we will download the datasets of interest, which have IDs 10, 11 and 12. 
We can do this using the Pennsieve API, but if you have downloaded the entire tutorial folder (not just the notebook) or if you are accessing it through Google Colab the relevant files as already available in the `res` directory. 

Only run the next code block if you do not already have the data sets available because it may take a while depending on your connection speed.

#### ⚠️  **SPARC Guru tip**: 
You can also browse the [SPARC Data Portal](https://sparc.science/data?type=dataset) on your web browser and download datasets to the `res` directory. 

In [ ]:
# The first three datasets are of interest so download them
get_dataset(dataId=10, version=3)
get_dataset(dataId=11, version=3)
get_dataset(dataId=12, version=3)

# Exploring downloaded dataset. 
# We need the derivative analysis result in derivative folder (we know this because we have inspected the dataset documentation in the manifest and README files)
print_folder_structure(dataId=10, version=3)
print_folder_structure(dataId=11, version=3)
print_folder_structure(dataId=12, version=3)

# copy the required files to res folder for further utilisation
!mkdir res
!mv Pennsieve-dataset-10-version-3/files/derivative/IGLE_data.xlsx res
!mv Pennsieve-dataset-11-version-3/files/derivative/IMA_analyzed_data.xlsx res
!mv Pennsieve-dataset-12-version-3/files/derivative/Efferent_data.xlsx res

### **Loading the 2D data**
In the 2D datasets that we are using, the distances are in percentages relative to an origin situated in the pyloric end of the stomach for the y-axis (left to right direction), and near the oesophagus for the z-axis (bottom to top direction). We are going to transform these coordinates to match those of the 3D stomach scaffold, in millimetres. For this, we are first going to define the boundaries in the z-axis and the y-axis. 

Here is a 2D representation of the data we are going to map to the 3D organ scaffold (retrieved from [1](https://sparc.science/datasets/10)).
![2d](img/2d_data_viz.png)

We will define a few helper functions to load the Excel files into the JupyterLab environment in the correct units:

In [ ]:
def get_position(percent, min_val, max_val):
    """ Converts the position from percentage to distance.
    
    Inputs:
    percent -- float, percentage value.
    min_val -- float, minimum distance for conversion.
    max_val -- float, maximum distance for conversion.
    
    Outputs:
    converted_value -- float, converted value.
    
    """
    return percent / 100 * (max_val - min_val) + min_val 

def load_data(data_name, col_keeps, y_lims, z_lims):
    """ Loads the data from an .xlsx file.
    
    Inputs:
    data_name -- str, nane of the .xlsx file to read.
    col_keeps -- dict{str:str}, dictionnary containing the names of the columns
        to keep.
    y_lims -- list[int], limits for the y direction to convert back to mm,
            first element is the minimum and second is the maximum.
    z_lims -- list[int], limits for the z direction to convert back to mm,
        first element is the minimum and second is the maximum.
    
    Outputs:
    df -- DataFrame, data frame containing the desired data.
    
    """
    df = pd.read_excel(data_name)
    # remove unnecessary columns
    for col in df.columns:
        if col in col_keeps:
            df.rename(columns = {col:col_keeps[col]}, inplace = True)
        else:
            df.drop(col, axis=1, inplace=True)
    df['y'] = get_position(df['%y'], y_lims[0], y_lims[1])
    df['z'] = get_position(df['%x'], z_lims[0], z_lims[1]) # x becomes z
    df['-%y'] = 100 - df['%y']
    # change the area to mm
    return df

We can now load the 2D spatial data (locations of the neurons) into DataFrames:

In [ ]:
# Define y-axis and z-axis boundaries based on geometry of 3D stomach scaffold
z_lims = [0, 36.7]
y_lims = [24.6, 0]

# fields to load from the Excel file
col_keeps = {'%x (distance from pylorus side)':'%x', '%y (distance from bottom)':'%y',
             'Average IGLE Area (um²)':'area', 'Area Of Innervation':'area', 
             'Neuron Area Of Innervation (um²) -Convex Hull':'area', 'V/D':'face',
             'specimen anatomical location':'face'}
df_igle = load_data('res/IGLE_data.xlsx', col_keeps, y_lims, z_lims)
df_ima = load_data('res/IMA_analyzed_data.xlsx', col_keeps, y_lims, z_lims)
df_efferent = load_data('res/Efferent_data.xlsx', col_keeps, y_lims, z_lims)

### **Preparing the 2D data**
Now that we have loaded the datasets into Python, we are going to prepare the data for mapping. For this, we are going to define a helper function. This will convert our data from the pandas DataFrame to a numpy array for easier manipulation:

In [ ]:
def convert_2D_data(df):
    """ Extracts required fields from the data frame to prepare for projection to 3D scaffold faces
    
    Inputs: 
    df -- pandas dataframe with at least, x, y, area and face fields
    
    Outputs:
    data_2_dim -- Nx2 numpy array two-dimensional data coordinates for the neurons
    data_intensity -- Nx1 numpy array for the area of innervation of each neuron to encode colour intensity
    data_face -- the face ont which the neurons must be projected (D for dorsal, V for ventral)    
    """
    print(df)
    data_array = np.array(df[['y','z','area','face']])

    # Convert input data to numpy coordinate array and intensity array.
    data_2_dim = np.array(data_array[:, (0,1)])
    data_intensity = data_array[:,2]
    data_face = data_array[:, 3]
    
    return data_2_dim, data_intensity, data_face

Now we will use the helper function to preprocess each dataset and extract the relevant fields:

In [ ]:
efferent_2D, efferent_intensity, efferent_face = convert_2D_data(df_efferent)
igle_2D, igle_intensity, igle_face = convert_2D_data(df_igle)
ima_2D, ima_intensity, ima_face = convert_2D_data(df_ima)

### **Loading the 3D mesh**
The 2D data is all set.  Time to focus on the 3D organ scaffold! Let's load the 3D stomach mesh provided in the `data` folder for this tutorial. 

#### ⚠️  **SPARC Guru tip**: 
However, it is possible to use the [**SPARC** Mapping Tools](https://docs.sparc.science/docs/map-core-scaffold-mapping-tools) to generate this mesh and convert it the STL file format. These tools are available as Python modules as well. We leave this task as an exercise for the reader.

In [ ]:
# Load the STL file
stomach_mesh = msh.Mesh.from_file('res/stom_surf_mesh.stl')

# adjust with coordinate axes (imported with signs reversed)
stomach_mesh.x -= np.min(stomach_mesh.x.flatten())
stomach_mesh.y -= np.min(stomach_mesh.y.flatten())
stomach_mesh.z -= np.min(stomach_mesh.z.flatten())

### **Mapping data**
We are now ready to map the 2D points to the 3D mesh.
The mathematics of projections are well understood, and more advanced methods to achieve this result are available in the [**SPARC** Mapping Tools](https://docs.sparc.science/docs/map-core-scaffold-mapping-tools). However, for this tutorial we will define the following simple helper function to project points perpendicular to their input 2D plane, onto the 3D stomach scaffold:

In [ ]:
def map_to_3D(input_pts, data_face, input_mesh):
    """ Maps an Nx2 numpy array of points in the y-z plane to have an
    x coordinate based on the nearest point in the y-z plane on the input mesh.
    Inputs:
    input_pts -- np.array(N, 2) with y coordinates in column 0, and
        z coordinates in column 1.
    data_face -- np.array(N,) of strings 'V' or 'D' indicating ventral or dorsal face respectively
    input_mesh -- numpy-stl mesh object with the target mesh.
    Outputs:
    out: np.array(N, 3), numpy array with x coordinates in column 0,
        y coordinates in column 1, and z coordinates in column 2.
    """
    # Create list of vertices from the vectors of each triangle in the mesh.
    vert = np.around(np.unique(input_mesh.vectors.reshape(
        [int(input_mesh.vectors.size/3), 3]), axis=0),2)

    # initial output array
    out = np.zeros((input_pts.shape[0], 3))

    # iterate over all points in y-z plane and find the closest mesh vertex in this plane
    for i, pt in enumerate(input_pts):
    
        if data_face[i] == 'V' or data_face[i] == 'Stomach - ventral':
            vert_candidate = vert[vert[:, 0] < 9.0, :]
            offset = 1
            jitter = 0.31
        else:
            vert_candidate = vert[vert[:, 0] > 8.7, :]
            offset = -1
            jitter = -0.31
            
        min_arg = np.argmin(np.sum(np.power(
            np.abs((pt-vert_candidate[:, 1:3])),2), 1))
        matched_x = vert_candidate[min_arg, 0]
        
        # Add some random movement so that triangles don't obscure points
        matched_x -= offset + np.random.rand()*jitter
        out[i, :] = [matched_x, pt[0], pt[1]]
    return out

Now we will use the helper function to project points for each of the datasets

In [ ]:
# Project to 3D stomach surface.
efferent_3D = map_to_3D(efferent_2D, efferent_face, stomach_mesh)
igle_3D = map_to_3D(igle_2D, igle_face, stomach_mesh)
ima_3D = map_to_3D(ima_2D, ima_face, stomach_mesh)

### **Visualising data**
The data has now been projected onto the 3D mesh. All that is left to do is visualise the mesh and the nerves! We are going to use the matplotlib package to visualise the data. However, there are **SPARC**-associated tools, such as [OpenCMISS](https://opencmiss.org) that provide a richer visualisation experience. 

#### ⚠️  **SPARC Guru tip**: 
Did you know that there are some **SPARC** tools that you can use to visualise the data? For this tutorial, we decided to keep everything within a Jupyter Lab environment. There are compatible Python APIs that could integrate the Scaffold Mapping Tool directly into this Jupyter Notebook but that process is more involved! Take a look [**SPARC** Mapping Tools](https://docs.sparc.science/docs/map-core-scaffold-mapping-tools) to read the documentation. 

In [ ]:
# Enable interactivity in jupyterlab.
%matplotlib widget 

def plotting_fct(mesh, efferent_3D, igle_3D, ima_3D, efferent_intensity, 
                 igle_intensity, ima_intensity, efferent, igle, ima):
    # Start a matplotlib 3d interactive figure.
    fig = plt.figure()
    fig.suptitle('Projection visualisation', fontsize=16)
    ax = fig.add_subplot(projection='3d')

    # Add mesh as triangle polygons to 3d matplotlib view.
    faces = mplot3d.art3d.Poly3DCollection(mesh.vectors, color=(0.960, 0.803, 0.650))
    faces.set_edgecolor((0.960, 0.803, 0.650, 0.1))
    faces.set_alpha(0.1)
    ax.add_collection3d(faces)

    # Plot the projected neurons coloured by their area of innervation.
    if efferent:
        ax.scatter(efferent_3D[:,0], efferent_3D[:,1], efferent_3D[:,2], 
                   c=efferent_intensity, cmap='Blues')
        
    if ima:
        ax.scatter(ima_3D[:,0], ima_3D[:,1], ima_3D[:,2], 
                   c=ima_intensity, cmap='PuRd')
        
    if igle:
        ax.scatter(igle_3D[:,0], igle_3D[:,1], igle_3D[:,2], 
                    c=igle_intensity, cmap='BuGn')        


    # Scale view to the mesh size and turn off axis chrome.
    ax.set_xlim(0,40)
    ax.set_ylim(-10,30)
    ax.set_zlim(-10,30)
    ax.axis('off')

    # Show ventral surface and plot projected neurons.
    ax.view_init(elev=10, azim=-57, vertical_axis='y')
    plt.show()

efferent=Checkbox(value=True, description="efferent")
igle=Checkbox(value=False, description="IGLE")
ima=Checkbox(value=False, description="IMA")
    
interact(plotting_fct, mesh=fixed(stomach_mesh), efferent_3D=fixed(efferent_3D), 
            igle_3D=fixed(igle_3D), ima_3D=fixed(ima_3D),
            efferent_intensity=fixed(efferent_intensity), 
            igle_intensity=fixed(igle_intensity), ima_intensity=fixed(ima_intensity), 
            efferent=efferent, igle=igle, ima=ima)    


### **Congratulations**
You have successfully completed your a Quilted Tutorial and are now on your way to becoming a **SPARC** Guru! 

We invite you to reuse this tutorial and explore the possibilities of using **SPARC** tools when possible. Try different things, such as adding ***your own code*** to generate a new stomach mesh with [Scaffold Maker](https://github.com/ABI-Software/scaffoldmaker). 

[1] T. L. Powley et al., “Spatial distribution and morphometric characterization of vagal afferents associated with the myenteric plexus of the rat stomach.” SPARC Consortium, 2019. doi: 10.26275/WZRY-SF7V.